In [ ]:
!pip install tb

In [ ]:
!pip install attrdict

In [ ]:
!pip install d3rlpy

In [3]:
import os
import re

import torch
import numpy as np
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import datasets
from torchvision import transforms
from torch import nn
from torch.nn import functional as F
import random
import torchvision.utils as vutils
import argparse
import math
import cv2
from matplotlib import pyplot as plt

from torchsummary import summary

from abc import abstractmethod

# Set random seed for reproducibility
seed = 1141
# seed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", seed)
random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:8" if (torch.cuda.is_available()) else "cpu")

Random Seed:  1141


In [31]:
train_ds = EdgesDataset(config=config)

dataloader = DataLoader(train_ds, config.parameters.dreamer.batch_size, shuffle=False)

In [41]:
train_ds[0]

(tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[1., 0., 1.,  ..., 0., 0., 0.],
          [1., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 1., 1., 1.]],
 
         [[0., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 1., 1.],
       

In [2]:
from encoder import Encoder

In [3]:
model = Encoder(config)

In [6]:
model.to('cuda')

Encoder(
  (_convs): Sequential(
    (0): Conv2dSame(2, 8, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2dSame(8, 16, kernel_size=(3, 3), stride=(2, 2))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2dSame(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Conv2dSame(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
  )
)

In [2]:
from decoder import Decoder

In [3]:
model = Decoder(config)

In [4]:
model.to('cuda')

Decoder(
  (_conv_transes): Sequential(
    (0): ConvTranspose2dSame(64, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): ConvTranspose2dSame(32, 16, kernel_size=(3, 3), stride=(2, 2))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): ConvTranspose2dSame(16, 8, kernel_size=(3, 3), stride=(2, 2))
    (7): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): ConvTranspose2dSame(8, 2, kernel_size=(3, 3), stride=(2, 2))
    (10): Sigmoid()
  )
)

In [1]:
from vae import EdgeEntropyVAE

In [3]:
model = EdgeEntropyVAE(config, 0.5)

In [4]:
model

EdgeEntropyVAE(
  (encoder): Encoder(
    (_convs): Sequential(
      (0): Conv2dSame(2, 8, kernel_size=(3, 3), stride=(2, 2))
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Conv2dSame(8, 16, kernel_size=(3, 3), stride=(2, 2))
      (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Conv2dSame(16, 32, kernel_size=(3, 3), stride=(2, 2))
      (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.01)
      (9): Conv2dSame(32, 64, kernel_size=(3, 3), stride=(2, 2))
      (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): LeakyReLU(negative_slope=0.01)
    )
  )
  (decoder): Decoder(
    (_conv_transes): Sequential(
      (0): ConvTranspose2dSame(64, 32, kernel_size=(3, 3), stride=(2, 2))
      (

In [10]:
model.reparameterize(torch.Tensor([0]), torch.Tensor([2]))

tensor([-0.3455])

In [15]:
torch.stack([torch.Tensor([[[0,1],[0,1]]]),torch.Tensor([[[1,1],[1,1]]])]).shape

torch.Size([2, 1, 2, 2])

In [54]:
import re
import cv2

def extract_metadata(fpath):
    
    with open(fpath, 'r') as file:
        info = file.read().rstrip()
        
    info_splited = info.split('_')
    
    timestep = int(info_splited[0])
    reward = int(info_splited[1])
    player_x = int(info_splited[2])
    player_y = int(info_splited[3])
    
    
    movements = info_splited[4].split(';')[:-1]
    
    walk_distance = jump_height = jump_true = 0
    
    for i in range(len(movements)):
        
        movement_list = [int(x) for x in list(movements[i])]
        
        if movement_list[0] == 1:
            walk_distance -= 1
            
        if movement_list[1] == 1:
            walk_distance += 1
        
        if movement_list[2] == 1:
            jump_true = 1
        
        # Если jumpt_true == 0, то не делаем ничего
        if (movement_list[3] == 0) and (jump_true == 1):
            jump_height += 1
        elif (movement_list[3] == 1) and (jump_true == 1):
            jump_height += 1
            jump_true = 0
    
    return timestep, reward, player_x, player_y, walk_distance, jump_height

def load_img(seed, timestep, config):
    
    image_path_1 = config.parameters.edges_dataset.data_root + f'/screenshots0000_{seed}/{timestep}.png'
    image_path_2 = re.sub(r'/screenshots/', r'/screenshots_add/', image_path_1)

    metadata_path = image_path_2[:-3] + 'txt'

    timestep, reward, player_x, player_y, walk_distance, jump_height = extract_metadata(metadata_path)

    if timestep < 7668:
        timestep_next = timestep + 6
        terminal = False
    else:
        terminal = True

    # do your magic here
    # флаг 0 = читаем черно-белое изображение
    img1 = cv2.imread(image_path_1, 0)
    img2 = cv2.imread(image_path_2, 0)

    img_full1 = transform_img(img1, dim=[self.config.image_size, self.config.image_size], zero_screenshot=self.zero_screenshot, aperture_size=self.config.aperture_size)
    img_full2 = transform_img(img2, dim=[self.config.image_size, self.config.image_size], zero_screenshot=self.zero_screenshot, aperture_size=self.config.aperture_size)

    img_part1 = transform_img(img1, dim=[self.config.image_size, self.config.image_size], zero_screenshot=self.zero_screenshot_part, part_size=self.config.part_size, player_x=player_x, player_y=player_y, aperture_size=self.config.aperture_size)
    img_part2 = transform_img(img2, dim=[self.config.image_size, self.config.image_size], zero_screenshot=self.zero_screenshot_part, part_size=self.config.part_size, player_x=player_x, player_y=player_y, aperture_size=self.config.aperture_size)

    sample_full = cv2.merge([img_full1, img_full2])
    sample_part = cv2.merge([img_part1, img_part2])
    
    return transforms.ToTensor()(sample_full), transforms.ToTensor()(sample_part), timestep, (player_x, player_y), (walk_distance, jump_height), reward, terminal

In [55]:
load_img(359803652, 12, config)

NameError: name 'transform_img' is not defined

In [3]:
train_ds = EdgesDataset(config=config)

dataloader = DataLoader(train_ds, config.parameters.dreamer.batch_size, shuffle=False)

In [4]:
seeds = get_seed_list(config.parameters.edges_dataset.data_root)

In [6]:
observations, actions, rewards, terminals = train_ds.load_episodes(seeds)

In [7]:
print(observations.shape, actions.shape, rewards.shape, terminals.shape)

(12770, 4, 64, 64) (12770, 2) (12770,) (12770,)


In [8]:
from d3rlpy.dataset import MDPDataset

dataset = MDPDataset(observations, actions, rewards, terminals, discrete_action=False)

In [16]:
dataset.dump('MDPdataset_SMF_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

In [13]:
encoder = Encoder(config).to(device)

In [15]:
observation = buffer.sample(10,10)['observation']
torch.cat([encoder(observation[:,:,0,:,:]),encoder(observation[:,:,1,:,:]),encoder(observation[:,:,2,:,:]),encoder(observation[:,:,3,:,:])],-1)

tensor([[[ 2.3416e-01,  6.0045e-01,  1.3020e+00,  ...,  1.1233e-01,
           1.1233e-01, -8.3058e-04],
         [ 3.3554e-01,  9.1756e-01,  6.0683e-01,  ..., -5.1583e-04,
          -5.1583e-04,  1.5307e-01],
         [-1.5669e-03,  1.1114e+00,  5.8742e-01,  ...,  1.2243e+00,
           2.1191e-01,  2.8043e+00],
         ...,
         [-5.0152e-03,  1.6649e+00,  5.2799e-01,  ..., -6.6693e-03,
          -6.6693e-03, -5.2427e-03],
         [ 3.0595e-01,  1.8360e+00, -1.6966e-03,  ...,  1.9619e+00,
           1.6654e+00,  1.0970e+00],
         [-7.8754e-03,  1.2732e+00, -4.6416e-03,  ..., -2.4606e-02,
          -4.2104e-04, -3.7070e-03]],

        [[ 1.9091e+00,  1.2435e-01,  2.9624e+00,  ...,  1.3341e+00,
          -1.0054e-03,  4.9796e-01],
         [ 1.9683e+00,  8.7909e-01,  4.6802e-01,  ...,  2.9038e+00,
           4.3718e-01,  1.3804e+00],
         [ 1.4281e-01,  7.9280e-01,  2.2420e-01,  ...,  1.7712e+00,
           3.2704e+00,  1.3540e+00],
         ...,
         [-3.9153e-03,  3

In [21]:
encoder(buffer.sample(10,10)['observation'][:,:,3,:,:]).shape

torch.Size([10, 10, 1024])

In [1]:
config_file='config.yml'

from dataset import EdgesDataset, get_seed_list
from utils import load_config
from torch.utils.data import DataLoader
import torch
import numpy as np

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

config = load_config(config_file)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from d3rlpy.dataset import MDPDataset

dataset = MDPDataset.load('MDPdataset_SMF_2023-03-24_13-47-05')

In [3]:
observation_shape = dataset.get_observation_shape()
action_size =  dataset.get_action_size()

In [4]:
from buffer import ReplayBuffer

In [5]:
buffer = ReplayBuffer(observation_shape, action_size, config)

In [6]:
for episode in dataset.episodes:
    for transition in episode.transitions:
        buffer.add(transition.observation, transition.action, transition.reward, transition.next_observation, transition.terminal)

In [7]:
buffer.buffer_index

12770

In [8]:
from encoder import Encoder

In [9]:
from dreamer import Dreamer

In [10]:
log_dir = (
        "/runs/"
        + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        + "_"
        + config.operation.log_dir
    )
writer = SummaryWriter(log_dir)
device = config.operation.device
obs_shape = dataset.get_observation_shape()
discrete_action_bool = False 
action_size = dataset.get_action_size()

agent = Dreamer(
            obs_shape, discrete_action_bool, action_size, writer, device, config
        )

In [11]:
agent.train_offline(buffer)

tensor([[[[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           ...,
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ..., 255., 255.,   0.],
           [  0.,   0.,   0.,  ...,   0., 255.,   0.]],

          [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [255., 255.,   0.,  ...,   0.,   0.,   0.],
           ...,
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

          [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           ...,
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [ 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (50x4296 and 1224x200)